In [ ]:
import azureml.core
from azureml.core import Workspace, Environment, Experiment, Datastore, Dataset, ScriptRunConfig

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)


In [ ]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel


In [ ]:
from datasets import load_dataset

dataset = load_dataset("yelp_review_full")
dataset["train"][100]

In [ ]:
dataset.shape

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [ ]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

In [ ]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch", num_train_epochs=1)

In [ ]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

exp = Experiment(ws, name="collect_metric")

In [ ]:
import time
import threading

from nvitop import ResourceMetricCollector
from nvitop import Device, GpuProcess, NA, colored

def collect_in_background(
    on_collect,
    collector=None,
    interval=None,
    *,
    on_start=None,
    on_stop=None,
    tag='',
    start=True,
):
    if collector is None:
        collector = ResourceMetricCollector()
    if interval is None:
        interval = collector.interval
    interval = min(interval, collector.interval)

    def target():
        if on_start is not None:
            on_start(collector)
        try:
            with collector(tag):
                try:
                    while on_collect(collector.collect()):
                        time.sleep(interval)
                except KeyboardInterrupt:
                    pass
        finally:
            if on_stop is not None:
                on_stop(collector)

    daemon = threading.Thread(target=target, daemon=True)
    if start:
        daemon.start()
    return daemon

In [ ]:
# base_metrics = collector.collect()
# metrics_vals = [x.split(' ')[0] for x in base_metrics.keys() if 'mean' in x and not 'load_average' in x]

from tqdm.notebook import tqdm

def set_tqdm():
    tempo_collector = ResourceMetricCollector()
    with tempo_collector(tag=''):
        base_metrics = tempo_collector.collect()
        
    metrics_vals = [x.split(' ')[0] for x in base_metrics.keys() if 'mean' in x and not 'load_average' in x]    
    print('================= METRICS =================')
    print(metrics_vals)
    print('================= ******* =================')

    tempo_collector.stop()

    tqdms = {}
    for x in metrics_vals:
        tqdms[x] = tqdm(total=0, position=2, bar_format='{desc}')
        tqdms[x].set_description_str(f"{x} - {0}")
    
    return tqdms

In [ ]:
from transformers.integrations import AzureMLCallback, MLflowCallback

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

trainer.callback_handler.remove_callback(MLflowCallback)

In [ ]:
li_logger = []

tqdms = set_tqdm()

run = exp.start_logging()

def on_collect(metrics):
    li_logger.append(metrics)
    for key in metrics.keys():
        if 'mean' in key and not 'load_average' in key:
            symbole = '%' if '%' in key else ''
            key_name = key.split(' ')[0]
            if key_name in tqdms:
                value = round(metrics[key], 2) if metrics[key] else 0
                run.log(key_name, value)
                tqdms[key_name].set_description_str(f"{key_name} - [{value} {symbole}]")
    return True

def on_stop(collector):
    print('logging is completed')

background_collector = ResourceMetricCollector()
daemon = collect_in_background(on_collect, background_collector, interval=5.0, on_stop=on_stop)

trainer.train()


In [ ]:
run.complete()